In [ ]:
%pip install ultralytics
%pip install boxmot
%pip install cv2
%pip install torch

import numpy as np
from ultralytics import YOLO
from boxmot import create_tracker
import cv2
import torch
from google.colab.patches import cv2_imshow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 14.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 63.4 MB/s eta 0:00:00
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110460 sha256=cbbdb298c8db26b4294513d79f0bce5b40b5b20d1c74095c41e5208d225b2821
  Stored in directory: /root/.cache/pip/wheels/77/bf/4c/b0c3f4798a0166668752312a67118b27a3cd341e13ac0ae6ee
Successfully built filterpy
  Attempting uninstall: regex
    Found 

In [ ]:
#Mount drive
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
# Navigate to Directory
%cd /content/drive/MyDrive/Colab\ Notebooks/EECS442/final_project/yolo_finetuning

/content/drive/MyDrive/Colab Notebooks/EECS442/final_project/yolo_finetuning


In [ ]:
MODEL = "./best.pt"
VIDEO = "./test.mp4"

In [ ]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#specify Yolo Model Fine-Tuned on labeled Roboter Data
detector = YOLO(MODEL)
detector.to(device)

#Create an OC-SORT tracker
tracker = create_tracker(
    tracker_type="ocsort",
)

cap = cv2.VideoCapture(VIDEO)


2025-12-03 23:34:57.783 | MainProcess/MainThread | INFO     | /usr/local/lib/python3.12/dist-packages/boxmot/trackers/basetracker.py:56 | __init__ - BaseTracker initialization parameters:
2025-12-03 23:34:57.784 | MainProcess/MainThread | INFO     | /usr/local/lib/python3.12/dist-packages/boxmot/trackers/basetracker.py:57 | __init__ - det_thresh: 0.6
2025-12-03 23:34:57.784 | MainProcess/MainThread | INFO     | /usr/local/lib/python3.12/dist-packages/boxmot/trackers/basetracker.py:58 | __init__ - max_age: 30
2025-12-03 23:34:57.784 | MainProcess/MainThread | INFO     | /usr/local/lib/python3.12/dist-packages/boxmot/trackers/basetracker.py:59 | __init__ - max_obs: 50
2025-12-03 23:34:57.785 | MainProcess/MainThread | INFO     | /usr/local/lib/python3.12/dist-packages/boxmot/trackers/basetracker.py:60 | __init__ - min_hits: 3
2025-12-03 23:34:57.785 | MainProcess/MainThread | INFO     | /usr/local/lib/python3.12/dist-packages/boxmot/trackers/basetracker.py:61 | __init__ - iou_threshold: 

In [ ]:
results = []

frame_idx = 1

with torch.inference_mode():
    while True:
        success, frame = cap.read()
        if not success:
            break

        # Run detection
        output = detector(frame, verbose=False)[0]

        # If there are detections:
        if output.boxes is not None and len(output.boxes) > 0:
            # xyxy: (N, 4) -> [x1, y1, x2, y2]
            boxes = output.boxes.xyxy.cpu().numpy()

            # confidence scores: (N,)
            scores = output.boxes.conf.cpu().numpy()

            # class indices: (N,)
            labels = output.boxes.cls.cpu().numpy()

            # Optional score threshold (similar to keep = scores >= 0.5)
            keep = scores >= 0.5
            boxes = boxes[keep]
            labels = labels[keep]
            scores = scores[keep]
            detections = np.concatenate([boxes, scores[:, None], labels[:, None]], axis=1)

        else:

            detections = np.empty((0, 6), dtype=float)

        # Update tracker and draw results
        #   INPUT:  M X (x, y, x, y, conf, cls)
        #   OUTPUT: M X (x, y, x, y, id, conf, cls, ind)
        res = tracker.update(detections, frame)


        # Log tracker results in MOT format
        for t in res:
            x1, y1, x2, y2, track_id, score, cls, _ = t
            w = x2 - x1
            h = y2 - y1

            results.append([
                frame_idx,
                int(track_id),
                float(x1),
                float(y1),
                float(w),
                float(h),
                float(score),
                int(cls),
            ])

        frame_idx += 1

        #tracker.plot_results(frame, show_trajectories=True)

        # Show output
        #cv2_imshow(frame)
        #if cv2.waitKey(1) & 0xFF == ord('q'):
        #    break

# Clean up
cap.release()
cv2.destroyAllWindows()


# Evaluation

In [ ]:
import pandas as pd
import csv

In [ ]:
df_results = pd.DataFrame(results, columns=['frame', 'track_id', 'x1', 'y1', 'w', 'h', 'score', 'class'])

In [ ]:
df_results.head(20)

,frame,track_id,x1,y1,w,h,score,class
0,1,4,1012.853760,165.688110,66.262939,109.739686,0.611936,0
1,1,3,1536.495117,704.941162,44.028808,62.834534,0.619521,0
2,1,2,383.067810,642.394226,62.229492,94.639710,0.707508,0
3,1,1,1468.304077,586.498169,97.584594,103.500123,0.732884,0
4,2,3,1536.442749,705.183899,44.165771,62.606628,0.622356,0
5,2,2,382.643494,640.624023,62.712128,96.525635,0.706708,0
6,2,1,1468.223145,586.479736,97.621094,103.412109,0.731741,0
7,3,3,1536.693970,704.123108,44.003906,63.745422,0.630546,0
8,3,2,382.613770,640.703369,62.731201,96.399902,0.706754,0
9,3,1,1468.223389,587.280884,97.583740,103.012146,0.727218,0


In [ ]:
csv_path = './results.csv'
header = ['frame', 'track_id', 'x1', 'y1', 'w', 'h', 'score', 'class']
with open(csv_path, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(header)     # write header row
    writer.writerows(results)   # write all data rows

print("Saved to:", csv_path)


Saved to: ./results.csv
